In [ ]:
#credits for this wonderful blog 
#https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

In [1]:
!pip install cmake 'gym[atari]' scipy


    100% |████████████████████████████████| 43.0MB 363kB/s ta 0:00:011    64% |████████████████████▋           | 27.6MB 10.6MB/s eta 0:00:02    69% |██████████████████████▍         | 30.0MB 3.6MB/s eta 0:00:04    74% |████████████████████████        | 32.2MB 5.0MB/s eta 0:00:03    79% |█████████████████████████▎      | 34.0MB 3.2MB/s eta 0:00:03
    100% |████████████████████████████████| 1.6MB 2.9MB/s ta 0:00:01
    100% |████████████████████████████████| 1.0MB 3.6MB/s ta 0:00:011
    100% |████████████████████████████████| 3.5MB 4.4MB/s ta 0:00:011
    100% |████████████████████████████████| 52.2MB 386kB/s ta 0:00:011    24% |████████                        | 12.9MB 2.9MB/s eta 0:00:14    40% |████████████▉                   | 20.9MB 6.0MB/s eta 0:00:06
  Running setup.py bdist_wheel for gym ... done
  Stored in directory: /Users/amirraza/Library/Caches/pip/wheels/87/e0/91/f56e44e8062f8cd549673da49f59e1d4fe8b17398119b1d221
Successfully built gym
spyder 3.3.2 requires pyqt5<5.10; pyth

In [3]:
import gym

env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [6]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [7]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [8]:
env.P[328]


{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [9]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [10]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 53.4 s, sys: 8.86 s, total: 1min 2s
Wall time: 58.1 s


In [11]:
q_table[328]


array([ -2.41095737,  -2.27325184,  -2.40884986,  -2.36169895,
       -10.75500648,  -9.5884607 ])

In [15]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100
frames= []

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1
        
        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.93
Average penalties per episode: 0.0


In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])#.getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : :_|
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

Timestep: 675
State: 197
Action: 2
Reward: -1
